# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


<div class="span5 alert alert-success">
<p>Your answers to Q1 and Q2 here</p>
</div>

In [5]:
w = data[data.race=='w']
b = data[data.race=='b']

In [6]:
w.call.value_counts()

0.0    2200
1.0     235
Name: call, dtype: int64

In [7]:
b.call.value_counts()

0.0    2278
1.0     157
Name: call, dtype: int64

### 1. What test is appropriate for this problem? Does CLT apply?

Based on exploring the data above, we see that we are dealing with proportions. Proportion of 'w' who got a call vs proportion of 'b' to got a call. And we are trying to see whether those proportions differ substastially and whether the difference is statistically significant. <br> <br>

 * Here we will have to calculate the z-score using the following formula:
     * Z = (( p1 - p2) - 0) / (p(1-p) (1/n1 +1/n2))**0.5
     * where p = ( y1 + y2 )/( n1 + n2) 
     
     Then, based on the z-value and using a confidence level of α = 0.05 level, we will be able to determine whether race has a significant impact on the rate of callbacks for resumes.
     

**The following conditions are required for CLT to be applicable:**
<br>
* Randomization Condition : The question promt suggests that the data collected is randomized, so this condition has been satisfied.
* Independence Assumption : Each resume is an independent entity and they are not interdependent on each other, so this condition has also been satisfied.
* 10% Condition: Here, we can assume that the data is acquired from large number of resumes, so we can consider that the sample size is less than 10% of the entire population.
* Sample Size Assumption: Sample size > 30, so the condition is satisfied
<br><br>
Since all the above conditions are satisfied, CLT is applicable in this data.

### 2. What are the null and alternate hypotheses?

> p1 = proportion of 'w' who received a call
> p2 = proportion of 'b' who received a call

* **H0 : p1 = p2 (Null Hypothesis)**
* **H1 : p1 != p2 (Alternate Hypothesis)**

### 3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.

In [8]:
w_counts=dict(w.call.value_counts())
b_counts=dict(b.call.value_counts())

y1 = w_counts[1] 
n1 = w_counts[0] + w_counts[1]
p1 = y1/n1

y2 = b_counts[1] 
n2 = b_counts[0] + b_counts[1]
p2 = y2/n2

def calc_z(y1,n1,p1,y2,n2,p2):
    p = ( y1 + y2 )/( n1 + n2)
    z = ( p1 - p2) / (p*(1-p)*((1/n1) +(1/n2)))**0.5
    p_value=stats.norm.sf(abs(z))*2
    return z,p_value

z_score, p_val = calc_z(y1,n1,p1,y2,n2,p2)
alpha = 0.05

print('Null Hypothesis : Race does not have a significant impact on the rate of callbacks for resumes. \np-value = ', p_val)
if p_val < alpha:  
    print("Verdict: The null hypothesis can be rejected.")
else:
    print("Verdict: The null hypothesis cannot be rejected.")

Null Hypothesis : Race does not have a significant impact on the rate of callbacks for resumes. 
p-value =  3.983886837585077e-05
Verdict: The null hypothesis can be rejected.


**For bootstrap analysis, we will assume that the null hypothesis is true and create permutation samples and find determine the statistics**

In [9]:
def permutation_sample(data1, data2):
    """Generate a permutation sample from two data sets."""
    data = np.concatenate((data1,data2))  # Concatenate the data sets: data
    permuted_data = np.random.permutation(data)  # Permute the concatenated array: permuted_data
    
    # Split the permuted array into two: perm_sample_1, perm_sample_2
    perm_sample_1 = permuted_data[:len(data1)]
    perm_sample_2 = permuted_data[len(data1):]

    return perm_sample_1, perm_sample_2


def draw_perm_reps(data_1, data_2, func, size=1):
    """Generate multiple permutation replicates."""

    # Initialize array of replicates: perm_replicates
    perm_replicates = np.empty(size)

    for i in range(size):
        # Generate permutation sample
        perm_sample_1, perm_sample_2 = permutation_sample(data_1,data_2)

        # Compute the test statistic
        perm_replicates[i] = func(perm_sample_1,perm_sample_2)

    return perm_replicates

def diff_in_props(rep_1, rep_2):
    """Compute the difference in proportions of two datasets"""
    count_1= np.count_nonzero(rep_1 == 1)
    count_2= np.count_nonzero(rep_2 == 1)
    prop1 = count_1/len(rep_1)
    prop2 = count_2/len(rep_2)
    return prop1-prop2

In [10]:
w_arr=w.call.values
b_arr=b.call.values
original_diff=diff_in_props(w_arr,b_arr)

reps=draw_perm_reps(w_arr,b_arr,diff_in_props, 100000)

p_val=np.sum(reps>=original_diff) /len(reps)

print('Null Hypothesis : Race does not have a significant impact on the rate of callbacks for resumes. \np-value = ', p_val)
if p_val < alpha:  
    print("Verdict: The null hypothesis can be rejected.")
else:
    print("Verdict: The null hypothesis cannot be rejected.")

Null Hypothesis : Race does not have a significant impact on the rate of callbacks for resumes. 
p-value =  0.0
Verdict: The null hypothesis can be rejected.


In [11]:
CI_95=np.percentile(reps,[2.5,97.5])
print('The 95% confidence interval of difference between the proportions is :', CI_95)

The 95% confidence interval of difference between the proportions is : [-0.01478439  0.01560575]


<div class="span5 alert alert-success">
<p> Your answers to Q4 and Q5 here </p>
</div>

### 4. Write a story describing the statistical significance in the context of the original problem.

We wanted to explore this data set to examine the level of racial discrimination in the United States labor market. The data was acquired by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers. In our analysis, we compared the two proportions - proportion of whites who revieved a call vs number of blacks who receive a call and the small p-vlaue suggested that there is indeed a difference in proportion of people who recieve a call based on their race.
<br>
This is a problem in the job market that needs to be overcome to provide equal opportunity for all the job seekers. Although jobs are supposed to hire regardless of the race, this data seems to suggest otherwise and indicates that race bias might still exist.

### 5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

In our analysis, we did find that the race determined based on name is a factor in callback success but we don't know if its the most important factor. There are several other factors in the dataset including - years of experience, education, etc which might also be playing an important role in callback success. Thus, all the other features would also have to be examined in order to determine which is the most important factor in callback success. 